<a href="https://colab.research.google.com/github/Podushka1337/homework/blob/main/SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

БД - SQLite. Халитова Елена, 09-313

In [2]:
import sqlite3
import os
from datetime import datetime

# Удаляем старую БД, если есть
if os.path.exists('library.db'):
    os.remove('library.db')

conn = sqlite3.connect('library.db')
cursor = conn.cursor()

# Создаём таблицу книг
cursor.execute('''
CREATE TABLE IF NOT EXISTS books (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    year_published INTEGER,
    genre TEXT
)
''')

# Добавляем тестовые данные
books_data = [
    ("The Great Gatsby", "F. Scott Fitzgerald", 1925, "Fiction"),
    ("1984", "George Orwell", 1949, "Dystopian"),
    ("To Kill a Mockingbird", "Harper Lee", 1960, "Classic")
]
cursor.executemany('''
INSERT INTO books (title, author, year_published, genre) VALUES (?, ?, ?, ?)
''', books_data)

# Выводим все книги
cursor.execute('SELECT * FROM books')
print("Все книги:")
for row in cursor.fetchall():
    print(row)

# Книги после 1950 года
cursor.execute('SELECT * FROM books WHERE year_published > 1950')
print("\nКниги после 1950:")
for row in cursor.fetchall():
    print(row)

# Названия на 'T'
cursor.execute('SELECT * FROM books WHERE title LIKE "T%"')
print("\nКниги на 'T':")
for row in cursor.fetchall():
    print(row)

# Сортировка по году
cursor.execute('SELECT * FROM books ORDER BY year_published ASC')
print("\nПо году издания:")
for row in cursor.fetchall():
    print(row)

# Первые 2 книги по названию
cursor.execute('SELECT * FROM books ORDER BY title LIMIT 2')
print("\nПервые 2 книги:")
for row in cursor.fetchall():
    print(row)

# Общее количество
cursor.execute('SELECT COUNT(*) FROM books')
total_books = cursor.fetchone()[0]
print(f"\nВсего книг: {total_books}")

# Группировка по жанрам
cursor.execute('SELECT genre, COUNT(*) FROM books GROUP BY genre')
print("\nКниг по жанрам:")
for row in cursor.fetchall():
    print(row)

# Обновляем год для "1984"
cursor.execute('UPDATE books SET year_published = 1948 WHERE title = "1984"')

# Удаляем "The Great Gatsby"
cursor.execute('DELETE FROM books WHERE title = "The Great Gatsby"')

# Индекс по авторам
cursor.execute('CREATE INDEX IF NOT EXISTS idx_author ON books(author)')

# Проверка индекса
cursor.execute("SELECT name FROM sqlite_master WHERE type='index' AND name='idx_author'")
index_exists = cursor.fetchone()
print(f"\nИндекс author: {'есть' if index_exists else 'нет'}.")

# Таблица логов и триггер
cursor.execute('''
CREATE TABLE IF NOT EXISTS logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    message TEXT
)
''')
cursor.execute('''
CREATE TRIGGER IF NOT EXISTS log_insert AFTER INSERT ON books
BEGIN
    INSERT INTO logs (message) VALUES ('Добавлена книга: ' || NEW.title);
END;
''')

# Транзакция с откатом при ошибке
try:
    conn.execute('BEGIN TRANSACTION')
    cursor.execute('INSERT INTO books (title, author, year_published, genre) VALUES (?, ?, ?, ?)',
                   ("New Book", "Author Name", 2023, "Fantasy"))
    cursor.execute('UPDATE books SET year_published = 1961 WHERE title = "To Kill a Mockingbird"')
    # raise Exception("Тест отката!")
    conn.commit()
except Exception as e:
    conn.rollback()
    print(f"Откат: {e}")

# Функция для возраста книги
def book_age(year_published):
    current_year = datetime.now().year
    return current_year - year_published

conn.create_function("book_age", 1, book_age)
cursor.execute('SELECT title, book_age(year_published) AS age FROM books')
print("\nВозраст книг:")
for row in cursor.fetchall():
    print(row)

# Представление modern_books
cursor.execute('''
CREATE VIEW IF NOT EXISTS modern_books AS
SELECT * FROM books WHERE year_published > 1950
''')

# Удаляем и проверяем таблицу books
cursor.execute('DROP TABLE IF EXISTS books')
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='books'")
table_exists = cursor.fetchone()
print(f"\nТаблица books: {'есть' if table_exists else 'удалена'}.")

# Восстанавливаем таблицу
cursor.execute('''
CREATE TABLE IF NOT EXISTS books (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    year_published INTEGER,
    genre TEXT
)
''')

# Снова добавляем данные
books_data = [
    ("The Great Gatsby", "F. Scott Fitzgerald", 1925, "Fiction"),
    ("1984", "George Orwell", 1949, "Dystopian"),
    ("To Kill a Mockingbird", "Harper Lee", 1960, "Classic")
]
cursor.executemany('''
INSERT INTO books (title, author, year_published, genre) VALUES (?, ?, ?, ?)
''', books_data)

# Составной индекс
cursor.execute('CREATE INDEX IF NOT EXISTS idx_author_year ON books(author, year_published)')
print("\nИндекс author+year создан.")

# Закрываем и удаляем БД
conn.close()
if os.path.exists('library.db'):
    os.remove('library.db')
    print("\nБД удалена.")
else:
    print("\nБД не найдена.")

Все книги:
(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction')
(2, '1984', 'George Orwell', 1949, 'Dystopian')
(3, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')

Книги после 1950:
(3, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')

Книги на 'T':
(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction')
(3, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')

По году издания:
(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction')
(2, '1984', 'George Orwell', 1949, 'Dystopian')
(3, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')

Первые 2 книги:
(2, '1984', 'George Orwell', 1949, 'Dystopian')
(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction')

Всего книг: 3

Книг по жанрам:
('Classic', 1)
('Dystopian', 1)
('Fiction', 1)

Индекс author: есть.
Откат: cannot start a transaction within a transaction

Возраст книг:

Таблица books: удалена.

Индекс author+year создан.

БД удалена.


**21 задание** (20 вариант)

In [1]:
import sqlite3
from abc import ABC, abstractmethod
from typing import List

DB_PATH = 'bikerental.db'

def init_db(db_path: str = DB_PATH):
    """Инициализация базы данных и создание таблицы bikes."""
    conn = sqlite3.connect(db_path)
    with conn:
        conn.execute("""
CREATE TABLE IF NOT EXISTS bikes (
    bike_id INTEGER PRIMARY KEY,
    model TEXT NOT NULL,
    rental_duration INTEGER NOT NULL,
    hourly_rate REAL NOT NULL,
    is_available INTEGER NOT NULL,
    type TEXT NOT NULL,
    battery_charge_fee REAL
)
""")
    conn.close()

# Исключения
class RentalServiceError(Exception):
    pass

class BikeNotFoundError(RentalServiceError):
    pass

class InvalidBikeDataError(RentalServiceError):
    pass

# Миксины
class LoggingMixin:
    def log(self, message: str):
        print(f"[LOG] {message}")

class NotificationMixin:
    def notify(self, message: str):
        print(f"[NOTIFICATION] {message}")

# Абстрактный класс Bike
class Bike(ABC):
    def __init__(self, bike_id: int, model: str, rental_duration: int,
                 hourly_rate: float, is_available: bool = True):
        if not model:
            raise InvalidBikeDataError("Модель не может быть пустой.")
        if rental_duration <= 0:
            raise InvalidBikeDataError("Продолжительность аренды должна быть положительным числом.")
        if hourly_rate <= 0:
            raise InvalidBikeDataError("Ставка аренды должна быть положительным числом.")
        self.__bike_id = bike_id
        self.__model = model
        self.__rental_duration = rental_duration
        self.__hourly_rate = hourly_rate
        self.__is_available = is_available

    @abstractmethod
    def calculate_rental_cost(self) -> float:
        pass

    def get_id(self) -> int:
        return self.__bike_id

    def get_model(self) -> str:
        return self.__model

    def set_model(self, model: str):
        if not model:
            raise InvalidBikeDataError("Модель не может быть пустой.")
        self.__model = model

    def get_rental_duration(self) -> int:
        return self.__rental_duration

    def set_rental_duration(self, duration: int):
        if duration <= 0:
            raise InvalidBikeDataError("Продолжительность аренды должна быть положительным числом.")
        self.__rental_duration = duration

    def get_hourly_rate(self) -> float:
        return self.__hourly_rate

    def set_hourly_rate(self, rate: float):
        if rate <= 0:
            raise InvalidBikeDataError("Ставка аренды должна быть положительным числом.")
        self.__hourly_rate = rate

    def is_available(self) -> bool:
        return self.__is_available

    def set_availability(self, available: bool):
        self.__is_available = available

    def __str__(self):
        return (f"Велосипед (ID {self.get_id()}): {self.get_model()}, "
                f"{self.get_rental_duration()} ч, {self.get_hourly_rate()} руб/ч")

    def __lt__(self, other):
        if not isinstance(other, Bike):
            return NotImplemented
        if self.get_hourly_rate() == other.get_hourly_rate():
            return self.get_rental_duration() < other.get_rental_duration()
        return self.get_hourly_rate() < other.get_hourly_rate()

    def __gt__(self, other):
        if not isinstance(other, Bike):
            return NotImplemented
        if self.get_hourly_rate() == other.get_hourly_rate():
            return self.get_rental_duration() > other.get_rental_duration()
        return self.get_hourly_rate() > other.get_hourly_rate()

class StandardBike(Bike):
    def calculate_rental_cost(self) -> float:
        return self.get_hourly_rate() * self.get_rental_duration()


class ElectricBike(Bike):
    def __init__(self, bike_id: int, model: str, rental_duration: int,
                 hourly_rate: float, battery_charge_fee: float,
                 is_available: bool = True):
        super().__init__(bike_id, model, rental_duration, hourly_rate, is_available)
        if battery_charge_fee < 0:
            raise InvalidBikeDataError("Сбор за заряд не может быть отрицательным.")
        self.__battery_charge_fee = battery_charge_fee

    def calculate_rental_cost(self) -> float:
        return self.get_hourly_rate() * self.get_rental_duration() + self.__battery_charge_fee

    def get_battery_charge_fee(self) -> float:
        return self.__battery_charge_fee

    def __str__(self):
        base = super().__str__()
        return f"{base}, сбор за заряд {self.__battery_charge_fee} руб"

class BikeRentalService(LoggingMixin, NotificationMixin):
    def __init__(self, db_path: str = DB_PATH):
        self.db_path = db_path
        init_db(self.db_path)

    def _connect(self):
        return sqlite3.connect(self.db_path)

    def add_bike(self, bike: Bike):
        if not isinstance(bike, Bike):
            raise InvalidBikeDataError("Можно добавлять только Bike.")
        conn = self._connect()
        with conn:
            conn.execute(
                "INSERT INTO bikes (bike_id, model, rental_duration, hourly_rate, is_available, type, battery_charge_fee) "
                "VALUES (?, ?, ?, ?, ?, ?, ?)",
                (
                    bike.get_id(), bike.get_model(), bike.get_rental_duration(),
                    bike.get_hourly_rate(), int(bike.is_available()),
                    type(bike).__name__,
                    getattr(bike, 'get_battery_charge_fee', lambda: None)()
                )
            )
        conn.close()
        self.log(f"Добавлен велосипед: {bike}")
        self.notify(f"Новый велосипед: {bike}")

    def edit_bike(self, bike_id: int, **kwargs):
        bikes = self.get_all_bikes()
        bike = next((b for b in bikes if b.get_id() == bike_id), None)
        if not bike:
            raise BikeNotFoundError(f"ID {bike_id} не найден.")
        if 'model' in kwargs:
            bike.set_model(kwargs['model'])
        if 'rental_duration' in kwargs:
            bike.set_rental_duration(kwargs['rental_duration'])
        if 'hourly_rate' in kwargs:
            bike.set_hourly_rate(kwargs['hourly_rate'])
        if 'is_available' in kwargs:
            bike.set_availability(kwargs['is_available'])
        conn = self._connect()
        with conn:
            conn.execute(
                "UPDATE bikes SET model=?, rental_duration=?, hourly_rate=?, is_available=? WHERE bike_id=?",
                (
                    bike.get_model(), bike.get_rental_duration(),
                    bike.get_hourly_rate(), int(bike.is_available()), bike_id
                )
            )
        conn.close()
        self.log(f"Изменен велосипед ID {bike_id}")
        self.notify(f"Велосипед обновлён: {bike}")

    def delete_bike(self, bike_id: int):
        conn = self._connect()
        with conn:
            cur = conn.execute("SELECT model FROM bikes WHERE bike_id=?", (bike_id,))
            row = cur.fetchone()
            if not row:
                raise BikeNotFoundError(f"ID {bike_id} не найден.")
            conn.execute("DELETE FROM bikes WHERE bike_id=?", (bike_id,))
        conn.close()
        self.log(f"Удалён велосипед ID {bike_id}")
        self.notify(f"Велосипед удалён ID {bike_id}")


    def get_all_bikes(self) -> List[Bike]:
        conn = self._connect()
        cur = conn.cursor()
        cur.execute("SELECT bike_id, model, rental_duration, hourly_rate, is_available, type, battery_charge_fee FROM bikes")
        rows = cur.fetchall()
        conn.close()
        bikes: List[Bike] = []
        for bike_id, model, duration, rate, avail, btype, fee in rows:
            if btype == 'ElectricBike':
                bikes.append(ElectricBike(bike_id, model, duration, rate, fee, bool(avail)))
            else:
                bikes.append(StandardBike(bike_id, model, duration, rate, bool(avail)))
        return bikes

    def search_by_model(self, substr: str) -> List[Bike]:
        conn = self._connect()
        cur = conn.cursor()
        pattern = '%' + substr.lower() + '%'
        cur.execute(
            "SELECT bike_id, model, rental_duration, hourly_rate, is_available, type, battery_charge_fee "
            "FROM bikes WHERE LOWER(model) LIKE ?", (pattern,)
        )
        rows = cur.fetchall()
        conn.close()
        result: List[Bike] = []
        for bike_id, model, duration, rate, avail, btype, fee in rows:
            if btype == 'ElectricBike':
                result.append(ElectricBike(bike_id, model, duration, rate, fee, bool(avail)))
            else:
                result.append(StandardBike(bike_id, model, duration, rate, bool(avail)))
        return result

    def analyze_data(self):
        bikes = self.get_all_bikes()
        total = len(bikes)
        available = sum(b.is_available() for b in bikes)
        avg_cost = sum(b.calculate_rental_cost() for b in bikes) / total if total else 0
        return {'total_bikes': total, 'available_bikes': available, 'average_cost': avg_cost}

if __name__ == "__main__":
    service = BikeRentalService()
    sb = StandardBike(1, "Giant", 3, 50.0)
    eb = ElectricBike(2, "E-BikeX", 5, 80.0, battery_charge_fee=15.0)
    service.add_bike(sb)
    service.add_bike(eb)
    service.edit_bike(1, rental_duration=4, hourly_rate=55.0)

    print("\nВсе велосипеды:")
    for bk in service.get_all_bikes():
        print(bk, "- Стоимость:", bk.calculate_rental_cost())

    print("\nПоиск по 'e-bi':")
    for bk in service.search_by_model("e-bi"):
        print(bk, "- Стоимость:", bk.calculate_rental_cost())

    print("\nСтатистика:", service.analyze_data())


[LOG] Добавлен велосипед: Велосипед (ID 1): Giant, 3 ч, 50.0 руб/ч
[NOTIFICATION] Новый велосипед: Велосипед (ID 1): Giant, 3 ч, 50.0 руб/ч
[LOG] Добавлен велосипед: Велосипед (ID 2): E-BikeX, 5 ч, 80.0 руб/ч, сбор за заряд 15.0 руб
[NOTIFICATION] Новый велосипед: Велосипед (ID 2): E-BikeX, 5 ч, 80.0 руб/ч, сбор за заряд 15.0 руб
[LOG] Изменен велосипед ID 1
[NOTIFICATION] Велосипед обновлён: Велосипед (ID 1): Giant, 4 ч, 55.0 руб/ч

Все велосипеды:
Велосипед (ID 1): Giant, 4 ч, 55.0 руб/ч - Стоимость: 220.0
Велосипед (ID 2): E-BikeX, 5 ч, 80.0 руб/ч, сбор за заряд 15.0 руб - Стоимость: 415.0

Поиск по 'e-bi':
Велосипед (ID 2): E-BikeX, 5 ч, 80.0 руб/ч, сбор за заряд 15.0 руб - Стоимость: 415.0

Статистика: {'total_bikes': 2, 'available_bikes': 2, 'average_cost': 317.5}
